In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import numpy as np

# 방법 1: 가중 평균을 통한 모델 파라미터 병합
class ModelAveraging:
    def __init__(self, model_class):
        self.model_class = model_class
    
    def average_models(self, model_paths, weights=None):
        """
        여러 모델의 파라미터를 가중평균으로 병합
        """
        if weights is None:
            weights = [1.0 / len(model_paths)] * len(model_paths)
        
        # 첫 번째 모델 로드
        averaged_model = self.model_class()
        first_state_dict = torch.load(model_paths[0], map_location='cpu')
        averaged_state_dict = OrderedDict()
        
        # 모든 파라미터 초기화
        for key in first_state_dict.keys():
            averaged_state_dict[key] = torch.zeros_like(first_state_dict[key])
        
        # 가중평균 계산
        for i, model_path in enumerate(model_paths):
            state_dict = torch.load(model_path, map_location='cpu')
            for key in state_dict.keys():
                averaged_state_dict[key] += weights[i] * state_dict[key]
        
        averaged_model.load_state_dict(averaged_state_dict)
        return averaged_model

# 방법 2: 계층적 앙상블 모델 (추천)
class HierarchicalEnsemble(nn.Module):
    def __init__(self, model_a_class, model_b_class, num_classes=17, vulnerable_classes=[3, 4, 7, 14]):
        super(HierarchicalEnsemble, self).__init__()
        self.num_classes = num_classes
        self.vulnerable_classes = vulnerable_classes
        
        # 분류기 A 앙상블 (전체 17개 클래스)
        self.models_a = nn.ModuleList([model_a_class() for _ in range(5)])
        
        # 분류기 B 앙상블 (취약클래스 4개)
        self.models_b = nn.ModuleList([model_b_class() for _ in range(5)])
        
        # 최종 융합을 위한 가중치
        self.alpha = nn.Parameter(torch.tensor(0.7))  # A의 가중치
        self.beta = nn.Parameter(torch.tensor(0.3))   # B의 가중치
        
    def load_pretrained_models(self, model_a_paths, model_b_paths):
        """사전 학습된 모델들 로드"""
        for i, path in enumerate(model_a_paths):
            self.models_a[i].load_state_dict(torch.load(path, map_location='cpu'))
            self.models_a[i].eval()
            
        for i, path in enumerate(model_b_paths):
            self.models_b[i].load_state_dict(torch.load(path, map_location='cpu'))
            self.models_b[i].eval()
    
    def forward(self, x):
        # 분류기 A 앙상블 예측
        preds_a = []
        for model in self.models_a:
            pred = F.softmax(model(x), dim=1)
            preds_a.append(pred)
        ensemble_a = torch.mean(torch.stack(preds_a), dim=0)
        
        # 분류기 B 앙상블 예측
        preds_b = []
        for model in self.models_b:
            pred = F.softmax(model(x), dim=1)
            preds_b.append(pred)
        ensemble_b = torch.mean(torch.stack(preds_b), dim=0)
        
        # 최종 예측 결합
        final_pred = torch.zeros_like(ensemble_a)
        
        # 모든 클래스에 대해 분류기 A의 예측 사용
        final_pred = self.alpha * ensemble_a
        
        # 취약클래스에 대해서는 분류기 B의 예측을 추가로 반영
        for i, cls_idx in enumerate(self.vulnerable_classes):
            final_pred[:, cls_idx] += self.beta * ensemble_b[:, i]
        
        return final_pred

# 방법 3: 지식 증류를 통한 단일 모델 생성
class KnowledgeDistillation:
    def __init__(self, student_model, teacher_models_a, teacher_models_b, 
                 vulnerable_classes=[3, 4, 7, 14], temperature=3.0):
        self.student = student_model
        self.teachers_a = teacher_models_a
        self.teachers_b = teacher_models_b
        self.vulnerable_classes = vulnerable_classes
        self.temperature = temperature
        
        # 교사 모델들을 평가 모드로 설정
        for teacher in self.teachers_a + self.teachers_b:
            teacher.eval()
    
    def get_teacher_predictions(self, x):
        """모든 교사 모델의 예측을 얻음"""
        with torch.no_grad():
            # 분류기 A 예측들
            preds_a = []
            for teacher in self.teachers_a:
                pred = F.softmax(teacher(x) / self.temperature, dim=1)
                preds_a.append(pred)
            ensemble_a = torch.mean(torch.stack(preds_a), dim=0)
            
            # 분류기 B 예측들
            preds_b = []
            for teacher in self.teachers_b:
                pred = F.softmax(teacher(x) / self.temperature, dim=1)
                preds_b.append(pred)
            ensemble_b = torch.mean(torch.stack(preds_b), dim=0)
            
        return ensemble_a, ensemble_b
    
    def distillation_loss(self, student_logits, teacher_preds_a, teacher_preds_b, 
                         true_labels=None, alpha=0.7, beta=0.3):
        """증류 손실 계산"""
        student_soft = F.softmax(student_logits / self.temperature, dim=1)
        
        # KL divergence 손실
        kl_loss_a = F.kl_div(F.log_softmax(student_logits / self.temperature, dim=1),
                            teacher_preds_a, reduction='batchmean')
        
        # 취약클래스에 대한 추가 증류
        vulnerable_mask = torch.zeros_like(student_soft)
        for cls_idx in self.vulnerable_classes:
            vulnerable_mask[:, cls_idx] = 1.0
        
        # 취약클래스 KL loss (분류기 B의 해당 인덱스와 매칭)
        kl_loss_b = 0
        for i, cls_idx in enumerate(self.vulnerable_classes):
            if i < teacher_preds_b.size(1):
                target_prob = teacher_preds_b[:, i].unsqueeze(1)
                student_prob = student_soft[:, cls_idx].unsqueeze(1)
                kl_loss_b += F.kl_div(torch.log(student_prob + 1e-8), 
                                    target_prob, reduction='batchmean')
        
        total_loss = alpha * kl_loss_a + beta * kl_loss_b
        
        # 실제 라벨이 있다면 하드 타겟 손실도 추가
        if true_labels is not None:
            hard_loss = F.cross_entropy(student_logits, true_labels)
            total_loss += 0.1 * hard_loss
        
        return total_loss

# 실제 사용 예시
def integrate_models():
    """모델 통합 실행 예시"""
    
    # 모델 경로들
    model_a_paths = [f'model_a_fold_{i}.pt' for i in range(5)]
    model_b_paths = [f'model_b_fold_{i}.pt' for i in range(5)]
    
    # 방법 1: 단순 가중평균 (분류기 A만)
    print("방법 1: 가중평균 모델 생성")
    averager = ModelAveraging(YourModelClass)  # YourModelClass를 실제 모델 클래스로 교체
    averaged_model_a = averager.average_models(model_a_paths)
    torch.save(averaged_model_a.state_dict(), 'averaged_model_a.pt')
    
    # 방법 2: 계층적 앙상블 (추천)
    print("방법 2: 계층적 앙상블 모델 생성")
    ensemble_model = HierarchicalEnsemble(YourModelClassA, YourModelClassB)
    ensemble_model.load_pretrained_models(model_a_paths, model_b_paths)
    torch.save(ensemble_model.state_dict(), 'hierarchical_ensemble.pt')
    
    # 방법 3: 지식 증류로 단일 모델 생성
    print("방법 3: 지식 증류 모델 학습 준비")
    # 이 경우 추가 학습 데이터와 학습 루프가 필요
    
    return ensemble_model

# 통합 모델 사용 예시
def use_integrated_model():
    """통합된 모델 사용 예시"""
    # 저장된 모델 로드
    model = HierarchicalEnsemble(YourModelClassA, YourModelClassB)
    model.load_state_dict(torch.load('hierarchical_ensemble.pt'))
    model.eval()
    
    # 예측
    with torch.no_grad():
        dummy_input = torch.randn(1, 3, 224, 224)  # 예시 입력
        output = model(dummy_input)
        predicted_class = torch.argmax(output, dim=1)
        
    return predicted_class

if __name__ == "__main__":
    # 실행
    integrated_model = integrate_models()
    print("모델 통합 완료!")

In [ ]:
# 사용 예시
model_a_paths = ['model_a_fold_0.pt', 'model_a_fold_1.pt', ...]
model_b_paths = ['model_b_fold_0.pt', 'model_b_fold_1.pt', ...]

ensemble = HierarchicalEnsemble(YourModelA, YourModelB)
ensemble.load_pretrained_models(model_a_paths, model_b_paths)

# 단일 파일로 저장
torch.save(ensemble.state_dict(), 'final_integrated_model.pt')